<a href="https://colab.research.google.com/github/sendmail123/DataScience/blob/master/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 37kB/s 
     |████████████████████████████████| 3.0MB 52.9MB/s 
     |████████████████████████████████| 419kB 74.3MB/s 


In [2]:
!pip install tqdm

In [3]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O ./cats_and_dogs_filtered.zip

--2020-01-10 20:57:19--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.118.128, 2404:6800:4003:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.118.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   112MB/s    in 0.6s    

2020-01-10 20:57:20 (112 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [4]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib .pyplot as plt
from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

'2.0.0-alpha0'

In [0]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [0]:
zip_object = zipfile.ZipFile(file=dataset_path,mode="r")

In [0]:
zip_object.extractall("./")

In [0]:
zip_object.close()

In [0]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [0]:
train_dir = os.path.join(dataset_path_new,"train")

In [0]:
validation_dir = os.path.join(dataset_path_new,"validation")

In [0]:
IMG_SHAPE = (128,128,3)

In [13]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,include_top=False, weights="imagenet")

9412608/9406464 [==============================] - 2s 0us/step


In [14]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalizationV1) (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [0]:
base_model.trainable=False

In [16]:
base_model.output

<tf.Tensor 'out_relu/Relu6:0' shape=(None, 4, 4, 1280) dtype=float32>

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [18]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(None, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(units=1,activation='sigmoid')(global_average_layer)

In [0]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalizationV1) (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),loss="binary_crossentropy",metrics=["accuracy"])

In [0]:
data_gen_train = ImageDataGenerator(rescale=1/255.0)
data_gen_valid = ImageDataGenerator(rescale=1/255.0)

In [24]:
train_generator = data_gen_train.flow_from_directory(train_dir,target_size=(128,128), batch_size=128,class_mode="binary")

Found 1000 images belonging to 2 classes.


In [29]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir,target_size=(128,128), batch_size=128,class_mode="binary")

Found 1000 images belonging to 2 classes.


In [30]:
model.fit_generator(train_generator,epochs=5,validation_data=valid_generator)

Epoch 1/5
8/8 [==============================] - 44s 5s/step - loss: 0.3085 - accuracy: 0.8670 - val_loss: 0.1007 - val_accuracy: 0.9640
Epoch 2/5
8/8 [==============================] - 7s 823ms/step - loss: 0.0326 - accuracy: 0.9990 - val_loss: 0.0811 - val_accuracy: 0.9670
Epoch 3/5
8/8 [==============================] - 7s 817ms/step - loss: 0.0155 - accuracy: 1.0000 - val_loss: 0.0789 - val_accuracy: 0.9660
Epoch 4/5
8/8 [==============================] - 7s 823ms/step - loss: 0.0084 - accuracy: 1.0000 - val_loss: 0.0698 - val_accuracy: 0.9720
Epoch 5/5
8/8 [==============================] - 7s 828ms/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 0.0634 - val_accuracy: 0.9740


In [0]:
valid_loss, valid_acuracy = model.evaluate_generator(valid_generator)

In [33]:
valid_acuracy

0.974